In [26]:
from qiskit.aqua.algorithms import VQE, NumPyEigensolver
import matplotlib.pyplot as plt
import tikzplotlib
import numpy as np
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.aqua.operators import Z2Symmetries
from qiskit import IBMQ, BasicAer, Aer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel
import os
import time
from qiskit.aqua.algorithms import NumPyEigensolver
from typing import List, NamedTuple
from qiskit.chemistry.transformations import FermionicTransformation, FermionicTransformationType, FermionicQubitMappingType
import pylab
import os
import time
from typing import List, NamedTuple

import matplotlib.pyplot as plt
import numpy as np
from IPython import display
from qiskit import QuantumCircuit
from qiskit.test.mock import FakeValencia
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.initial_states import Zero
from qiskit.circuit.library import EfficientSU2
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.aqua.operators import PauliExpectation
from qiskit.aqua.operators import Z2Symmetries
from qiskit.aqua.operators.converters import AbelianGrouper
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.circuit import Parameter
from qiskit.providers.aer import QasmSimulator, AerProvider
from qiskit.providers.aer.noise import NoiseModel
from qiskit.quantum_info import Statevector
from qiskit.test.mock import *
from qiskit.visualization import plot_error_map

In [22]:
plt.rcParams["figure.dpi"] = 100
plt.style.use("seaborn-pastel")

In [3]:
basis_sets = ['sto6g', '631g', '321g']
threshold = 0.00000001

In [4]:
def get_qubit_hamiltonian(d, basis_set, map_type='jordan_wigner'):
    """
    Obtain the qubit operators from the fermionic operators after second quantization.
    Args:
        d: inter-atomic distance
        basis_set: computational chemistry basis set
        map_type: encoding method

    Returns: qubit Hamiltonian as a weighted summed of Paulis, energy shift because of nuclear repulsion, number of particles and number of spin orbitals

    """
    driver = PySCFDriver(atom="H .0 .0 .0; H .0 .0 " + str(d) ,
                         unit=UnitsType.ANGSTROM,
                         charge=0,
                         spin=0,
                         basis=basis_set)
    molecule = driver.run()
    h1 = molecule.one_body_integrals
    h2 = molecule.two_body_integrals
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2
    repulsion_energy = molecule.nuclear_repulsion_energy
    fermionic_op = FermionicOperator(h1,h2)
    qubit_op = fermionic_op.mapping(map_type=map_type, threshold=threshold)
    return qubit_op, repulsion_energy, num_particles, num_spin_orbitals

In [17]:
qubit_ops = {}
for set in basis_sets:
    qubit_ops["qubitOp_{0}".format(set)],_,_,_ = get_qubit_hamiltonian(0.735, set)

for label in qubit_ops:
    print("\n --- {} ---".format(label),qubit_ops[label])



 --- qubitOp_sto6g --- Representation: paulis, qubits: 4, size: 15

 --- qubitOp_631g --- Representation: paulis, qubits: 8, size: 185

 --- qubitOp_321g --- Representation: paulis, qubits: 8, size: 185


In [6]:
bond_lengths = np.linspace(0.1, 3.5, 50)

In [10]:
def store_intermediate_result(eval_count, parameters, mean, std):
    counts = []
    values = []
    parameters = []
    counts.append(eval_count)
    values.append(mean)
    parameters.append(parameters)
    return counts, values, parameters

In [11]:
aqua_globals.random_seed = 500

In [28]:
def construct_qinstance(device):
    simulator_backend = Aer.get_backend('qasm_simulator')

    device_backend = FakeValencia()
    qubit_connectivity = device_backend.configuration().coupling_map
    noise_model = NoiseModel.from_backend(device_backend)

    if device == "qasm_ideal":
        qinstance = QuantumInstance(backend=simulator_backend,
                                    seed_simulator=aqua_globals.random_seed)

    elif device == "fake_valencia":
        qinstance = QuantumInstance(backend=device_backend,
                                    noise_model=noise_model,
                                    coupling_map=qubit_connectivity,
                                    seed_simulator=aqua_globals.random_seed,
                                    seed_transpiler=aqua_globals.random_seed)

    elif device == "fake_valencia_with_mitigation":
        qinstance = QuantumInstance(backend=device_backend, seed_simulator=aqua_globals.random_seed,
                                    seed_transpiler=aqua_globals.random_seed,
                                    noise_model=noise_model,
                                    measurement_error_mitigation_cls=CompleteMeasFitter,
                                    cals_matrix_refresh_period=30)
    return qinstance

In [13]:
optimizers = [COBYLA, SPSA, SLSQP]

In [55]:
exact_energies = []
vqe_energies_effSU2 = []
vqe_energies_uccsd = []

for dist in bond_lengths:
    qubitOp, shift, num_particles, num_spin_orbitals = get_qubit_hamiltonian(dist,basis_set='sto6g')
    result = NumPyEigensolver(qubitOp).run()
    exact_energies.append(np.real(result.eigenvalues) + shift)
    initial_state = HartreeFock(num_spin_orbitals,
                                num_particles,
                                two_qubit_reduction=False,
                                qubit_mapping='jordan_wigner')
    optimizer = SPSA(maxiter=200)

    ansatz = [UCCSD(num_orbitals= num_spin_orbitals,
                    num_particles= num_particles,
                    initial_state= initial_state,
                    qubit_mapping='jordan_wigner',
                    two_qubit_reduction=False,),
              EfficientSU2(num_qubits=qubitOp.num_qubits,
                           su2_gates=['rz','rx','rz'],
                           initial_state=Zero(qubitOp.num_qubits))]
    for psi in ansatz:
        print(psi)
        vqe = VQE(qubitOp, psi, optimizer)
        vqe_result = np.real(vqe.run(construct_qinstance('qasm_ideal'))['eigenvalue'] + shift)
        if psi == ansatz[0]:
            vqe_energies_uccsd.append(vqe_result)
        elif psi == ansatz[1]:
            vqe_energies_effSU2.append(vqe_result)
        print("Interatomic Distance:", np.round(dist, 2), "VQE Result:", vqe_result, "Exact Energy:", exact_energies[-1])


Interatomic Distance: 0.1 VQE Result: 2.6966398249842958 Exact Energy: [2.68713685]
     ┌──────────┐┌──────────┐ ┌──────────┐                    ┌───────────┐»
q_0: ┤ RZ(θ[0]) ├┤ RX(θ[4]) ├─┤ RZ(θ[8]) ├──■────■─────────■──┤ RZ(θ[12]) ├»
     ├──────────┤├──────────┤ ├──────────┤┌─┴─┐  │         │  └───────────┘»
q_1: ┤ RZ(θ[1]) ├┤ RX(θ[5]) ├─┤ RZ(θ[9]) ├┤ X ├──┼────■────┼────────■──────»
     ├──────────┤├──────────┤┌┴──────────┤└───┘┌─┴─┐┌─┴─┐  │        │      »
q_2: ┤ RZ(θ[2]) ├┤ RX(θ[6]) ├┤ RZ(θ[10]) ├─────┤ X ├┤ X ├──┼────────┼──────»
     ├──────────┤├──────────┤├───────────┤     └───┘└───┘┌─┴─┐    ┌─┴─┐    »
q_3: ┤ RZ(θ[3]) ├┤ RX(θ[7]) ├┤ RZ(θ[11]) ├───────────────┤ X ├────┤ X ├────»
     └──────────┘└──────────┘└───────────┘               └───┘    └───┘    »
«     ┌───────────┐┌───────────┐                                         »
«q_0: ┤ RX(θ[16]) ├┤ RZ(θ[20]) ├───────────────────■────────■─────────■──»
«     ├───────────┤├───────────┤┌───────────┐    ┌─┴─┐      │         │  

QiskitError: 'Keyboard interrupt in parallel_map.'

In [ ]:
pylab.plot(distances, energies[j], label=algorithms[j])
pylab.xlabel('Interatomic distance')
pylab.ylabel('Energy')
pylab.title('H2 Ground State Energy')
pylab.legend(loc='upper right');

In [ ]:
pylab.plot(distances, np.subtract(hf_energies, energies[1]), label='Hartree-Fock')
pylab.plot(distances, np.subtract(energies[0], energies[1]), label='VQE')
pylab.xlabel('Interatomic distance')
pylab.ylabel('Energy')
pylab.title('Energy difference from NumPyMinimumEigensolver')
pylab.legend(loc='upper left');